In [148]:
from Bio import AlignIO
#Alignment conducted with multiple alignment tool @ http://www.ebi.ac.uk/Tools/msa/clustalo/
aln = AlignIO.read('cpec-align.clustal', 'clustal')

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_dna
from Bio import SeqIO
from Bio.Alphabet import IUPAC


In [108]:
aln[0].id = "L16"
aln[1].id = "L11"
aln[2].id = "L1"
aln[3].id = "L6"
aln[4].id = "L3"
aln[5].id = "L4"

L10 = aln[7] #sequence read with forward primer not used because of poor quality
aln[6].id = "vic"

In [109]:
"""
Find good start and end indices to trim mutant sequences at
"""

def is_good(index, sequence):
    good = 0
    for next_base in sequence[index+1:index+100]: #100 chosen as large enough to ensure good 
        if next_base != "-":
            good += 1      
    if good > 98:
        return True
def find_end(index, sequence):
    for more_index, more_base in enumerate(sequence[index:]):
        if more_base == "-" and is_good(more_index, sequence[index:]): #Find first non-isolated occurrence of "-" (a single occurence indicates a frameshift mutation)
            return more_index
    return False
            
start_list = []
end_list = []

for sequence in aln[0:6]:
    for index, base in enumerate(sequence):
        if base != "-":
            if is_good(index, sequence):
 
                start_list.append(index)
                if find_end(index, sequence) != False:
                    end_list.append(index + find_end(index, sequence))

                break

from collections import Counter
data = Counter(end_list)


end = data.most_common(1)[0][0] 
start = max(start_list)       


In [110]:
"""
Complete full PB2 sequences for each mutant by combining with template sequence
"""
vic_end = find_end(end, aln[6]) #Stripping the - at the end of the template sequence - NOT USED

full_seq_dict = {}
for sequence in aln[0:6]:
    full_seq = str(aln[6].seq)[0:start] + str(sequence.seq)[start:end] + str(aln[6].seq)[end:]
    full_seq_dict[sequence.id] = full_seq.replace("-", "") #remove gaps resulting from deletions or insertions or frameshift mutations
print(full_seq_dict)

{'L16': 'AGCAAAAGCAGGTCAATTATATTCAGTATGGAAAGAATAAAAGAACTACGGAATCTGATGTCGCAGTCTCGCACTCGCGAGATACTGACAAAAACCACAGTGGACCATATGGCCATAATTAAGAAGTACACATCGGGGAGACAGGAAAAGAACCCGTCACTTAGGATGAAATGGATGATGGCAATGAAATATCCAATCACTGCTGACAAAAGGGTAACAGAAATGGTTCCGGAGAGAAATGAACAAGGACAAACTCTATGGAGTAAAATGAGTGATGCTGGATCAGATAGAGTGATGGTATCACCTTTGGCTGTAACATGGTGGAATAGGAATGGACCCGTGACAAGTACGGTCCATTACCCAAAAGTGTACAAAACTTATTTCGACAAAGTCGAAAGGTTAAAACATGGAACCTTTGGCCCTGTCCATTTCAGAAATCAAGTCAAGATACGCAGAAGAGTAGACATAAACCCTGGTCATGCAGACCTCAGTGCCAAAGAGGCACAAGATGTAATTATGGAAGTTGTTTTTCCCAATGAAGTGGGAGCCAGAATACTAACATCAGAATCACAACTAACAATAACTAAGGAGAAAAAAGAAGAGCTCCGAGATTGCAAAATTTCTCCCTTGATGGTCGCATACATGTTAGAGAGAGAACTTGTACGAAAAACAAGATTTCTCCCAGTTGCTGGCGGAACAAGCAGTATATACATTGAAGTTTTACATTTGACTCAGGGAACGTGTTGGGAACAAATGTACACTCCAGGTGGAGGAGTGAGGAATGACGATGTTGACCAAAGCCTAATTATTGCGGCCAGGAACATAGTAAGAAGAGCCGCAGTATCAGCAGATCCACTAGCATCTTTATTGGAGATGTGCCACAGCACGCAAATTGGCGGAACAAGGATGGTGGACATTCTTAGACAGAACCCGACTGAAGAACAAGCTGTGGATATATGCAAGGCTGCAATGGGATTGAGAATCAGCTCAT

In [111]:
"""
Translate sequences
"""

full_protein_dict = {}
for name, sequence in full_seq_dict.items():
    coding_dna = Seq(sequence, generic_dna)
    full_protein_dict[name] = str(coding_dna.translate())
    
print(full_protein_dict)

strip_vic = str(aln[6].seq).replace("-", "")
seq_strip_vic = Seq(strip_vic, generic_dna)
vic_trans = seq_strip_vic.translate()
print(vic_trans)


{'L16': 'SKSRSIIFSMERIKELRNLMSQSRTREILTKTTVDHMAIIKKYTSGRQEKNPSLRMKWMMAMKYPITADKRVTEMVPERNEQGQTLWSKMSDAGSDRVMVSPLAVTWWNRNGPVTSTVHYPKVYKTYFDKVERLKHGTFGPVHFRNQVKIRRRVDINPGHADLSAKEAQDVIMEVVFPNEVGARILTSESQLTITKEKKEELRDCKISPLMVAYMLERELVRKTRFLPVAGGTSSIYIEVLHLTQGTCWEQMYTPGGGVRNDDVDQSLIIAARNIVRRAAVSADPLASLLEMCHSTQIGGTRMVDILRQNPTEEQAVDICKAAMGLRISSSFSFGGFTFKRTSGSSVKKEEEVLTGNLQTLRIRVHEGYEEFTMVGKRATAILRKATRRLVQLIVSGRDEQSIAEAIIVAMVFSQEDCMIKAVRGDLNFVNRANQRLNPMHQLLRHFQKDAKVLFQNWGVEHIDSVMGMVGVLPDMTPSTEMSMRGIRVSKMGVDEYSSTERVVVSIDRFLRVRDQRGNVLLSPEEVSETQGTERLTITYSSSMMWEINGPESVLVNTYQWIIRNWEAVKIQWSQNPAMLYNKMEFEPFQSLVPKATRSQYSGFVRTLFQQMRDVLGTFDTAQIIKLLPLQLLHRSEAECSSLN*L*M*GDQG*EYL*GAILLYSTTTKPVKG*QFSEKMPAL*LKTQMKAHQEWSPPS*EGSSL*VRKTGDMDQH*ASMN*VTLQKGKRLMC*SGKETWCW**NENGNSSILTDSQTATKRIRMAIN*C*IV*KRPCFY', 'L3': 'SKSRSIIFSMERIKELRNLMSQSRTREILTKTTVDHMAIIKKYTSGRQEKNPSLRMKWMMAMKYPITADKRVTEMVPERNEQGQTLWSKMSDAGSDRVMVSPLAVTWWNRNGPVTSTVHYPKVYKTYFDKVERLKHGTFGPVHFRNQVKIRRRVDINPGHADLSAKEAQDVIMEVVFPNEVGARILTSESQLTITKEKKEE

In [112]:
"""
Align translated sequences
"""

#Used Clustal Omega Multiple Alignment because I still haven't downloaded ClustalW yet

'\nAlign translated sequences\n'

In [161]:
"""
Identify number of mutations in each sequence
"""

aln_aa = (AlignIO.read('cpec-protein-align-auto.clustal', 'clustal')) #I can't access the .seq and .id attributes with parse

import math

#arbitrarily decided to opt for max proportion of mutant sequence
protein_start = int(math.floor(start/3)) 
protein_end = int(math.ceil(end/3))

def is_diff(mutant, vic):
    num_mut = 0
    where_mut = []
    for aa_index, aa in enumerate(mutant[protein_start:protein_end]):
        if aa != vic[aa_index+protein_start]: 
            num_mut += 1
            where_mut.append(aa_index+protein_start)
    return (num_mut, where_mut)
def is_bad(index, sequence):
    bad = 0
    for next_aa in sequence[index+1:index+100]: #100 chosen as large enough to ensure good 
        if next_base != "-":
            good += 1      
    if good > 98:
        return True

output_handle = open("cpec-proteins-forward.fasta", "w")

mut_dict = {}

"""
Save sequences in a SeqRecord and write to a file
"""

for sequence in aln_aa[0:6]:
    #[0]:number of mutations, [1]: location of mutations
    mut_dict[sequence.id] = is_diff(sequence.seq, aln_aa[6].seq)[0], is_diff(sequence.seq, aln_aa[6].seq)[1]
    mut_str = str(mut_dict[sequence.id])

    record = SeqRecord(Seq(str(sequence.seq), IUPAC.protein), id=sequence.id, name="cpec-"+sequence.id+"-forward",description = "(number of mutations, [locations of mutations]) " + mut_str)
    SeqIO.write(record, output_handle, "fasta")
output_handle.close()

print(mut_dict)
print("")
print("Length of sequence of interest:", len(vic_trans[protein_start:protein_end])) #number of amino acids in sequence of interest



{'L16': (112, [596, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 653, 654, 655, 656, 657, 658, 660, 661, 662, 663, 664, 665, 667, 668, 672, 673, 674, 675, 676, 677, 678, 679, 680, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 728, 731, 732, 733, 734, 735, 736, 737, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751]), 'L3': (1, [597]), 'L4': (4, [627, 690, 732, 763]), 'L1': (2, [638, 727]), 'L6': (5, [567, 577, 610, 692, 746]), 'L11': (1, [576])}

Length of sequence of interest: 206
